In [1]:
from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import copy

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances #We can calculate this matrix using 2 for loops, 
#but this isn't that important to calculate so we directly use this
import matplotlib.pyplot as plt

In [7]:
def test(data, cutoff, linkage):
    #This is done using dynamic programming approach
    # if 1, it is single linkage else 2 is complete linkage, 3 is average linkage
    distance_matrix = euclidean_distances(data, data) #Step 1 - Calculate distance matrix
    #distance_matrix = np.array([np.linalg.norm(data - p, ord=1, axis=1) for p in data])
    print("dist", distance_matrix)
    
    distance_matrix = np.tril(distance_matrix) #Step 2 - Since matrix is symmetric, we just keep lower triangle matrix
    distance_matrix[distance_matrix == 0] = np.inf #Step 3 - Replace 0 by inf, it makes it easy for us to extract minimum using min function
    df = pd.DataFrame(data=np.ones(data.shape[0])*np.inf) #Initialized a dataframe which will store which point is in which cluster
    #if cutoff > distance_matrix.shape[0]: #If user provides impractical cut-off, cluster everthing into one cluster and not listen to user 
    #    cutoff = distance_matrix.shape[0]
    if linkage == 1: #This 1 means formula of single linkage will be used, it is explained ahead
        d = {} #This dictionary keeps record of which data points or cluster are merging, hence can be used to make a dendogram
        i=0
        distances = np.array([np.linalg.norm(data - p, ord=1, axis=1) for p in data])
        while True:
        #for i in range(0,cutoff):
            ij_min = np.unravel_index(distance_matrix.argmin(), distance_matrix.shape) #from the distance matrix, get the minimum distance
            #np.unravel_index gives us the position of minimum distance. e.g. (0,4) is where minimum value is present in matrix.
            #This is what we need as in Hierarchical clustering, we merge the two pairs with minimum distance
            print(ij_min[0],ij_min[1])
            if ij_min[0] == 166 or ij_min[1] == 166:
                print("here")
            if i == 0:
                df.iloc[ij_min[0]] = 0
                df.iloc[ij_min[1]] = 0
            else:
                try:
                    a = int(df.iloc[ij_min[0]])
                except:
                    df.iloc[ij_min[0]] = i
                    a = i
                try:
                    b = int(df.iloc[ij_min[1]])
                except:
                    df.iloc[ij_min[1]] = i
                    b = i
                df[(df[0]==a) | (df[0]==b)] = i
            d[i] = ij_min
            #The if, else code till here is just filling the dataframe as the two points/clusters combine.
            #So, for example if 1 and 2 combines, dataframe will have 1 : 0, 2 : 0. Which means point 1 and 2 both are in same cluster (0th cluster)
            for j in range(0, ij_min[0]):
                #we want to ignore the diagonal, and diagonal is 0. We replaced 0 by infinte. 
                #So this if condition will skip diagonals
                if np.isfinite(distance_matrix[ij_min[0]][j]) and np.isfinite(distance_matrix[ij_min[1]][j]):
                    #after two points/cluster are linked, to calculate new distance we take minimum distance for single linkage
                    distance_matrix[ij_min[1]][j] = min(distance_matrix[ij_min[0]][j], distance_matrix[ij_min[1]][j])
            # To avoid the combined data points/cluster in further calculations, we make them infinte.
            #Our if loop above this, will therefore skip the infinite record entries.
            distance_matrix[ij_min[0]] = np.inf
            print(distance_matrix)
            i += 1
            
            link = single_link(distances)
            distances = merge(link, distances)
            if len(distances) == cutoff:
                break
            
        print("i=",i)
        print(df[0])
        return d, df[0].to_numpy()
    
def single_link(distances):
    d = copy.deepcopy(distances)
    d = np.where(d == 0, np.inf, d)
    return np.unravel_index(d.argmin(), d.shape)

def merge(link, distances):
  n = distances.shape[0]
  m = np.array([distances[link[0]], distances[link[1]]])
  m = np.min(m, axis=0)
  m = np.expand_dims(m[m != 0], 0)
  # Delete two rows
  distances = np.delete(distances, link, axis=0)
  # Delete two columns
  distances = np.delete(distances, link, axis=1)
  distances = np.reshape(distances, (n -2, n-2))
  # Add a column
  distances = np.c_[distances, m.T]
  m = np.expand_dims(np.append(m, 0),0)
  # Add a row
  distances = np.r_[distances, m]
  return distances

In [98]:
x = np.array([[0.8,0.7],[-0.1,0.2],[0.9,0.8],[0.0,0.2],[0.2,0.1]])
test(x,2,1)

3 1
[[inf inf inf inf inf]
 [1.3 inf inf inf inf]
 [0.2 1.6 inf inf inf]
 [inf inf inf inf inf]
 [1.2 0.4 1.4 0.3 inf]]
2 0
[[inf inf inf inf inf]
 [1.3 inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]
 [1.2 0.4 1.4 0.3 inf]]
4 3
[[inf inf inf inf inf]
 [1.3 inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]
 [inf inf inf inf inf]]
i= 3
0    1.0
1    2.0
2    1.0
3    2.0
4    2.0
Name: 0, dtype: float64


({0: (3, 1), 1: (2, 0), 2: (4, 3)}, array([1., 2., 1., 2., 2.]))

In [5]:
from scipy.io import arff

abalone = arff.loadarff("abalone.arff")
abalone_df = pd.DataFrame(abalone[0])
abalone_df = (abalone_df - abalone_df.min()) / (abalone_df.max() - abalone_df.min())
abalone_np = np.array(abalone_df)

In [8]:
a,b=test(abalone_np, 5,1)

dist [[0.         0.59292665 0.46572414 ... 0.87456562 0.53555772 0.71906032]
 [0.59292665 0.         0.63289343 ... 1.30019588 0.97115359 0.96554651]
 [0.46572414 0.63289343 0.         ... 0.72211217 0.43977261 0.3658285 ]
 ...
 [0.87456562 1.30019588 0.72211217 ... 0.         0.38114241 0.46138794]
 [0.53555772 0.97115359 0.43977261 ... 0.38114241 0.         0.33977566]
 [0.71906032 0.96554651 0.3658285  ... 0.46138794 0.33977566 0.        ]]
100 1
[[       inf        inf        inf ...        inf        inf        inf]
 [0.58548962        inf        inf ...        inf        inf        inf]
 [0.46572414 0.63289343        inf ...        inf        inf        inf]
 ...
 [0.87456562 1.30019588 0.72211217 ...        inf        inf        inf]
 [0.53555772 0.97115359 0.43977261 ... 0.38114241        inf        inf]
 [0.71906032 0.96554651 0.3658285  ... 0.46138794 0.33977566        inf]]
133 1
[[       inf        inf        inf ...        inf        inf        inf]
 [0.58548962        in

In [101]:
boo, co = np.unique(b, return_counts=True)

In [102]:
boo

array([143., 185., 192., 194.,  inf])

In [103]:
abalone_np[165]

array([1.        , 0.98876404, 0.78947368, 1.        , 1.        ,
       0.89148737, 0.71859296, 0.58823529])

In [105]:
co

array([  2,  41, 143,  13,   1], dtype=int64)

In [88]:
co

array([  2,  41, 143,  13,   1], dtype=int64)

In [89]:
x

array([[ 0.8,  0.7],
       [-0.1,  0.2],
       [ 0.9,  0.8],
       [ 0. ,  0.2],
       [ 0.2,  0.1]])

In [90]:
x[1,:]=float('inf')

In [91]:
x

array([[0.8, 0.7],
       [inf, inf],
       [0.9, 0.8],
       [0. , 0.2],
       [0.2, 0.1]])

In [94]:
x[:,1]=float('inf')

In [95]:
x

array([[0.8, inf],
       [inf, inf],
       [0.9, inf],
       [0. , inf],
       [0.2, inf]])